# Simulating Constitutive Processes of semantic change within heterogeneous populations of speakers

In [3]:
# basic imports
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

# project code imports
from mod.agent import *
from mod.plot import *
# from mod.network import *

## A very basic simulation

In [5]:
vocab_size = 10
semantic_features = 3

In [6]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

#### Simple dyadic interaction across repeated turns in a random environment

In [7]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [8]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(85.0878)

In [9]:
# and set the number of iterations
turns = 300

In [10]:
utt_tracking, vocab_dif = [], []

In [11]:
for _ in tqdm(range(turns)):
    env = starting_env.sample()
    speaker_prob = torch.rand(size=(1,))

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)

    else:
        utt = ag2.speak(env, lam=3)

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]


  0%|          | 0/300 [00:00<?, ?it/s]

In [12]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [13]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(9.2615)

In [14]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

#### Simple dyadic interaction in random environment but with introduction of new term by one of the speakers tailor made to a particular feature.

In [15]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [16]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(94.4283)

In [17]:
# and set the number of iterations
turns = 300
add_vocab_in = 10

In [18]:
vocab_dif = []

In [19]:
for rd in tqdm(range(turns)):
    env = starting_env.sample()
    speaker_prob = torch.rand(size=(1,))

    new_vocab_round = ((rd % add_vocab_in) == 0) * (rd != 0)

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)


    else:
        utt = ag2.speak(env, lam=3)

    if new_vocab_round:
        ag1.add_vocab_item()
        ag2.add_vocab_item()

        utt = ag1.vocab.shape[0] - 1

        f = (env / env.sum().unsqueeze(-1)).argmax()

        # if speaker 1s turn to talk, update their mental lexicon
        if speaker_prob > .5:
            ag1.vocab[utt][f] = env[0,f]
            ag1.var[utt] = torch.FloatTensor([1e-5]*ag1.var.shape[-1])
            ag1.var[utt][f] = .05

        # if speaker 2s turn to talk, update their mental lexicon
        else:
            ag2.vocab[utt][f] = env[0,f]
            ag2.var[utt] = torch.FloatTensor([1e-5]*ag2.var.shape[-1])
            ag2.var[utt][f] = .05

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]

  0%|          | 0/300 [00:00<?, ?it/s]

In [20]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [21]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(42.1067)

In [22]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment with novel words introduced',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

## Randomly changing environment

In [41]:
vocab_size = 10
semantic_features = 3
new_environment_prob = .25

In [42]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

#### Simple dyadic interaction across repeated turns in a random environment

In [43]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [44]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(54.6626)

In [45]:
# and set the number of iterations
turns = 300

In [46]:
utt_tracking, vocab_dif = [], []

In [47]:
for _ in tqdm(range(turns)):
    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    speaker_prob = torch.rand(size=(1,))

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)

    else:
        utt = ag2.speak(env, lam=3)

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]


  0%|          | 0/300 [00:00<?, ?it/s]

In [48]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [49]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(1.2948)

In [50]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

#### Simple dyadic interaction in random environment but with introduction of new term by one of the speakers tailor made to a particular feature.

In [52]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [53]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(63.3160)

In [54]:
# and set the number of iterations
turns = 300
add_vocab_in = 10

In [55]:
vocab_dif = []

In [56]:
for rd in tqdm(range(turns)):
    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    speaker_prob = torch.rand(size=(1,))

    new_vocab_round = ((rd % add_vocab_in) == 0) * (rd != 0)

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)


    else:
        utt = ag2.speak(env, lam=3)

    if new_vocab_round:
        ag1.add_vocab_item()
        ag2.add_vocab_item()

        utt = ag1.vocab.shape[0] - 1

        f = (env / env.sum().unsqueeze(-1)).argmax()

        # if speaker 1s turn to talk, update their mental lexicon
        if speaker_prob > .5:
            ag1.vocab[utt][f] = env[0,f]
            ag1.var[utt] = torch.FloatTensor([1e-5]*ag1.var.shape[-1])
            ag1.var[utt][f] = .05

        # if speaker 2s turn to talk, update their mental lexicon
        else:
            ag2.vocab[utt][f] = env[0,f]
            ag2.var[utt] = torch.FloatTensor([1e-5]*ag2.var.shape[-1])
            ag2.var[utt][f] = .05

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]

  0%|          | 0/300 [00:00<?, ?it/s]

In [57]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [58]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(13.3233)

In [59]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment with novel words introduced',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

## Stochastic changes to environment

In [60]:
vocab_size = 10
semantic_features = 3
new_environment_prob = .25

In [61]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

#### Simple dyadic interaction across repeated turns in a random environment

In [62]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [63]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(36.9516)

In [64]:
# and set the number of iterations
turns = 300

In [65]:
utt_tracking, vocab_dif = [], []

In [66]:
for _ in tqdm(range(turns)):
    # new_env_prob = torch.rand(size=(1,))
    # if new_env_prob > new_environment_prob:
    #     starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    speaker_prob = torch.rand(size=(1,))

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)

    else:
        utt = ag2.speak(env, lam=3)

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]


  0%|          | 0/300 [00:00<?, ?it/s]

In [67]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [68]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(0.7262)

In [69]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

#### Simple dyadic interaction in random environment but with introduction of new term by one of the speakers tailor made to a particular feature.

In [118]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [119]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(72.4732)

In [120]:
# and set the number of iterations
turns = 300
add_vocab_in = 50

In [121]:
vocab_dif = []

In [122]:
for rd in tqdm(range(turns)):
    # new_env_prob = torch.rand(size=(1,))
    # if new_env_prob > new_environment_prob:
    #     starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env
    speaker_prob = torch.rand(size=(1,))

    new_vocab_round = ((rd % add_vocab_in) == 0) * (rd != 0)

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)


    else:
        utt = ag2.speak(env, lam=3)

    if new_vocab_round:
        ag1.add_vocab_item()
        ag2.add_vocab_item()

        utt = ag1.vocab.shape[0] - 1

        f = (env / env.sum().unsqueeze(-1)).argmax()

        # if speaker 1s turn to talk, update their mental lexicon
        if speaker_prob > .5:
            ag1.vocab[utt][f] = env[0,f]
            ag1.var[utt] = torch.FloatTensor([1e-5]*ag1.var.shape[-1])
            ag1.var[utt][f] = .05

        # if speaker 2s turn to talk, update their mental lexicon
        else:
            ag2.vocab[utt][f] = env[0,f]
            ag2.var[utt] = torch.FloatTensor([1e-5]*ag2.var.shape[-1])
            ag2.var[utt][f] = .05

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]

  0%|          | 0/300 [00:00<?, ?it/s]

In [123]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [124]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(64.4827)

In [125]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment with novel words introduced',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

## Random and stochastic changes to environment

In [126]:
vocab_size = 10
semantic_features = 3
new_environment_prob = .25

In [127]:
starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

#### Simple dyadic interaction across repeated turns in a random environment

In [128]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [129]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(98.9815)

In [130]:
# and set the number of iterations
turns = 300

In [131]:
utt_tracking, vocab_dif = [], []

In [132]:
for _ in tqdm(range(turns)):
    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env

    speaker_prob = torch.rand(size=(1,))

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)

    else:
        utt = ag2.speak(env, lam=3)

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]


  0%|          | 0/300 [00:00<?, ?it/s]

In [133]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [134]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(3.3136)

In [135]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

#### Simple dyadic interaction in random environment but with introduction of new term by one of the speakers tailor made to a particular feature.

In [152]:
ag1 = agent(vocab_size,semantic_features, starting_observations=5)
ag2 = agent(vocab_size,semantic_features, starting_observations=5)

In [153]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(52.8011)

In [154]:
# and set the number of iterations
turns = 300
add_vocab_in = 50

In [155]:
vocab_dif = []

In [156]:
for rd in tqdm(range(turns)):
    new_env_prob = torch.rand(size=(1,))
    if new_env_prob > new_environment_prob:
        starting_env = torch.distributions.MultivariateNormal(torch.randn(size=(1,semantic_features)), covariance_matrix=torch.eye(semantic_features) * .2)

    env = starting_env.sample()
    starting_env.loc = env
    speaker_prob = torch.rand(size=(1,))

    new_vocab_round = ((rd % add_vocab_in) == 0) * (rd != 0)

    if speaker_prob > .5:
        utt = ag1.speak(env, lam=3)


    else:
        utt = ag2.speak(env, lam=3)

    if new_vocab_round:
        ag1.add_vocab_item()
        ag2.add_vocab_item()

        utt = ag1.vocab.shape[0] - 1

        f = (env / env.sum().unsqueeze(-1)).argmax()

        # if speaker 1s turn to talk, update their mental lexicon
        if speaker_prob > .5:
            ag1.vocab[utt][f] = env[0,f]
            ag1.var[utt] = torch.FloatTensor([1e-5]*ag1.var.shape[-1])
            ag1.var[utt][f] = .05

        # if speaker 2s turn to talk, update their mental lexicon
        else:
            ag2.vocab[utt][f] = env[0,f]
            ag2.var[utt] = torch.FloatTensor([1e-5]*ag2.var.shape[-1])
            ag2.var[utt][f] = .05

    ag2.listen(utt, env)
    ag1.listen(utt, env)
    vocab_dif += [((ag1.vocab - ag2.vocab)**2).sum()]

  0%|          | 0/300 [00:00<?, ?it/s]

In [157]:
# utt_tracking = torch.FloatTensor(utt_tracking)
vocab_dif = torch.FloatTensor(vocab_dif)

In [158]:
((ag1.vocab - ag2.vocab)**2).sum()

tensor(1.6541)

In [159]:
fig = plot(vocab_dif.numpy(), 'vocabulary difference')
fig.update_layout(
    title='Dyadic interaction in a random environment with novel words introduced',
    yaxis_title='Δ P(w|m)',
    xaxis_title='turns'
)
fig.show()

## Within a social network

In [ ]:
no_agents = 50
no_connections = 10

## Returning to forced birth vs. pro-life

So this one is trolly and fun. Basically, we want to replicate the changes in frequency for forced birth (FB) versus pro-life (PL) across months prior to and after the Dobbs decision. We can have a set of features representing the relative probability that a word will be associated with a feature. Something like the following table (note: these aren't normalized probabilities in the example below. I'm not sure whether we ought to do that or not.):

| **Date range** | **Antiabortion** | **legality** | **($\neg$) activist** | **morality** |
|------------|--------------| -------- | ----------------- | -------- |
| _2022/1-2022/5_ | .35          | .2       | .45            |  .0001   |
| _2022/6-2023/1_ | .2           | .45      | .35            | .0001    |
| ... | ... | ... | ... | ... |
| _2024/1-2024/5_ | .0001 | .2  | .45 | .35 |

which we can then use as a series of environments that dictate (1) what people say, (2) how people update their beliefs on the constraints around when to use certain words. We can even initialize the network with the same number of "users" as there are on _r/Feminism_!
